# SARS-CoV-2 Exploration

This tutorial will take us through two paths of exploring some details about the SARS-CoV-2 virus and the resulting COVID-19 pandemic. We will first look at how we can use Python to explore the virus sequence. We will then visualize the epidemiological data (i.e. how many cases are there and where they are). 

Some of the Python here is a bit advanced, especially if you have never used Python so far. Don't worry. Just take it in for now. As your knowledge of Python grows, you will see these are all tools you can learn to use yourself. 

### Acknowledgement and resources

This tutorial is based on work done by:
- Chris Rands [biopython-coronavirus](https://github.com/chris-rands/biopython-coronavirus)
- Harshit Tyagi [COVID-19 Dashboard](https://github.com/dswh/voila-covid-19-dashboard/blob/master/notebooks/covid_19_dashboard.ipynb)

This tutorial makes extensive use of [Biopython](https://biopython.org/), a Python software project that has created lots of functions that make working with biological information more easy. Check the linked website to learn more about that project. 

First, there are some software tools that need to be installed. This next line uses [pip](https://pypi.org/project/pip/) the package installer for python. 

**Tip**: This will take a few minutes. While this is running you will see an asterisk ( "In [\*]"). Wait until a number (i.e. "In [1]" ) appears, meaning the process has completed. 

In [ ]:
# install a few missing libraries this will take a few minutes

!pip3 install plotly==4.9.0 seaborn==0.9.1 folium==0.11.0 

Since we have just installed some new libraries you may need to restart Python - Go to the **Kernel** menu and select **Restart**. You will get a warning that this will erase all variables and that's OK. select **Restart**. 

In [ ]:
# load needed libraries 

import os
import matplotlib
import Bio as Bio 
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import plotly
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

## Looking at SARS-CoV-2 Sequence

We can use BioPython to download the SARS-CoV-2 Reference Genome from NCBI [Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2?report=fasta). this next code will fetch the genome above. 

In [ ]:

# get text genbank record for SARS-CoV-2
Entrez.email = "Your.Name.Here@example.org" # we have to provide NCBI an email - you 
handle = Entrez.efetch(db="nucleotide", # download from the nucleotide database
                       id="NC_045512", # get the sequence ID
                       rettype="gb", # get the record in genbank format
                       retmode="text") # show the results as text
print(handle.read())


We will now get the sequence in a format that Biopython can read and save it as a file on our computer. We will also ask Python to print the sequence out for us. 

In [ ]:
filename = "NC_045512.fasta"
Entrez.email = "A.N.Other@example.com"
if not os.path.isfile(filename):
    # Downloading...
    net_handle = Entrez.efetch(
        db="nucleotide", 
        id="NC_045512", 
        rettype="genbank", 
        retmode="fasta"
    )
    out_handle = open(filename, "w")
    out_handle.write(net_handle.read())
    out_handle.close()
    net_handle.close()
    

print("Parsing...")
sars_cov2_refseq = SeqIO.read(filename, "fasta")
print(sars_cov2_refseq)

We can view the fasta file using the 'cat' command. This is a linux command (note it starts with "!")

In [ ]:
!cat NC_045512.fasta

Using Python we now have a variable caled `sars_cov2_refseq` which has certian properties

In [ ]:
print(sars_cov2_refseq)

We can use some functions in BioPython to explore the sequence

In [ ]:
sars_cov2_refseq.id # show the ID of the sequence 

In [ ]:
str(sars_cov2_refseq.seq) # show the entire sequence

In [ ]:
# get the nucleotide length
print("Sequence length (bp)", len(sars_cov2_refseq)) 

In [ ]:
# get the GC content
print("GC content (%)", GC(sars_cov2_refseq.seq))

We can transcribe the sequence into RNA

In [ ]:
str(sars_cov2_refseq.seq.transcribe())

We can translate the sequence into protein

In [ ]:
str(sars_cov2_refseq.seq.translate())

We can also look for specific sequences within the sequence. For example, what if we wanted to find a motif "TTAAGC"

In [ ]:
#We can look to "find" in the "seq" (sequence)
sars_cov2_refseq.seq.find("ATGTTTGTTTTTCTTGTTTTA")

We can put those coordinates to see the sequence

In [ ]:
sars_cov2_refseq.seq[21562:21583]

Let's download some related viruses to make some comparisons. In the paper [Selection of viral variants during persistent infection of insectivorous
bat cells with Middle East respiratory syndrome coronavirus](https://www.nature.com/articles/s41598-020-64264-1), researchers explored the similarities and differences between SARS-CoV-2 and a related virus, MERS-CoV (a corona virus that leads to Middle Eastern Respiratory Syndrome) in infected bat cells. We will download the reference MERS and another SARS-CoV sequence. 



In [ ]:
#Middle East Respiratory Syndrome-related coronavirus (MERS-CoV)
#Severe Acute Respiratory Syndrome-related coronavirus (SARS-CoV)


virus_dict = {"NC_038294":"NC_038294.fasta", #MERS Betacoronavirus England 1, complete genome
           "NC_004718.3":"NC_004718.3.fasta" #SARS coronavirus Tor2, complete genome
          }


for virus in virus_dict.keys():
    filename = virus_dict[virus]
    Entrez.email = "A.N.Other@example.com"
    if not os.path.isfile(filename):
        # Downloading...
        net_handle = Entrez.efetch(
            db="nucleotide", id=virus, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        


Let's see the files we have dowloaded

In [ ]:
!ls *.fasta

let's load all sequences into one file so we can make some comparisons

In [ ]:
!cat NC_045512.fasta NC_038294.fasta NC_004718.3.fasta > viruses.fasta

We now have all 3 viral genomes in a single file

In [ ]:
!cat viruses.fasta

We can dowload and use a software called MUSCLE to make an alignment of all three sequences to more easily spot similarities and diferences. 

In [ ]:
# Download and install MUSCLE 

# Linux 
!wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz && tar -xvf muscle3.8.31_i86linux64.tar.gz

Now we can align the viruses (this will take a few minutes). Check out the NCBI SARS-CoV-2 [sequence tree](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/precomptree) while you are waiting

In [ ]:
!./muscle3.8.31_i86linux64 -in viruses.fasta -out alignment.txt -clw

We can view the alignment here (first 10 lines of file), but it may be better to use the Jupyter browser to open the file in a new tab. 

In [ ]:
!head alignment.txt

Let's view a phylogenetic tree of this result. First here is the alignment in Biopython format

In [ ]:
from Bio import AlignIO
align = AlignIO.read("alignment.txt", "clustal")
print(align)

According to the [NCBI map](https://www.ncbi.nlm.nih.gov/projects/sviewer/?id=NC_045512&tracks=[key:sequence_track,name:Sequence,display_name:Sequence,id:STD649220238,annots:Sequence,ShowLabel:false,ColorGaps:false,shown:true,order:1][key:gene_model_track,name:Genes,display_name:Genes,id:STD3194982005,annots:Unnamed,Options:ShowAllButGenes,CDSProductFeats:true,NtRuler:true,AaRuler:true,HighlightMode:2,ShowLabel:true,shown:true,order:9]&v=1:29903&c=null&select=null&slim=0), the spike protein is at about 21,563..25,384bp. Let's see this in the alignment. 

In [ ]:

print(str(align[:,21562:21583]))

This is not quite right because the sequence actually begins with "ATGTTTGTTTTTCTTGTTTTA". We can search to see where this ended up in the alignment. 

In [ ]:
align[1].seq.find('ATGTTTGTTTTT')

We can take this location and add the length of the spike protein.

In [ ]:
print(str(align[:,22806:26627]))

This is close, but every "-" move our alignment a bit, so lets find the last few bases...

In [ ]:
align[1].seq.find('AATTACATTACACATAA')

In [ ]:
print(str(align[:,22806:27043]))

We can save those sequences to do a search...

In [ ]:
sars_cov2_spike = str(align[1].seq[22806:27043])

We can use this to do a BLAST search

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

#blast the sequence result using blastn, against the
#nr database, evalue of 0.0001, return top 5 hits
blast_result = NCBIWWW.qblast('blastn','nt',sars_cov2_spike, expect = 0.0001, hitlist_size=5) 

blast_record = NCBIXML.read(blast_result)

In [ ]:
# print the blast hit results

for records in blast_record.alignments:
    print(records)

We can move on to make a tree with muscle...

In [ ]:
!./muscle3.8.31_i86linux64  -maketree -in viruses.fasta -out viruses.phy  -cluster neighborjoining

And visualize with BioPython. (Tip, open the viruses.phy file and shorten the name to make the naming more clear)

In [ ]:
from Bio import Phylo
tree = Phylo.read("viruses.phy", "newick")
tree.rooted = False
Phylo.draw(tree)

# Exploring epidemiological data

John Hopkins University aggregates and organizes [COVID-19 data](https://github.com/CSSEGISandData) we can easily use. We'll import it below

In [ ]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

We can build a table that shows us each of these datasets

In [ ]:
confirmed_df.head()

In [ ]:
recovered_df.head()

In [ ]:
death_df.head()

In [ ]:
country_df.head()

There are a series of steps we can do to clean the data (changing column names, and reorganizing the data

In [ ]:
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

country_df.head()

We will sum the numbers from all countries and create them in new dataframes (spreadsheets)

In [ ]:
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

Let's look at the total stats

In [ ]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

We can summarise these data in a widget to make them interactive (and pretty). Here is. the top 10

In [ ]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

We will sort the countries for the next visualization by confirmed cases

In [ ]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [ ]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

We can use the `folium` widget to plot our cases on a map. 

In [ ]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

If you look carefully you will see the data need to be cleaned a little more. Probably some entries in the dataset are  not completely joining states and territories into one sum. 